In [1]:
import zipfile
import pandas as pd
import os
# Path to your zip file
zip_path = '../data/raw/MachineLearningRating_v3.zip'

# Path to the .txt file inside the extracted folder
txt_file = '../data/raw/MachineLearningRating_v3.txt'

# Check if the .txt file exists before extracting the zip file
if not os.path.exists(txt_file):
    # Extract the zip file if the .txt file does not exist
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('../data/raw')  # Extract to the raw data folder
else:
    print(f"The file {txt_file} already exists.")

# Load the data assuming pipe separator
df = pd.read_csv(txt_file, sep='|')

# Display the first few rows of the data to confirm it's loaded
print(df.head())

/tmp/ipykernel_2504505/3691039534.py:19: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(txt_file, sep='|')


   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0               145249     12827  2015-03-01 00:00:00             True   
1               145249     12827  2015-05-01 00:00:00             True   
2               145249     12827  2015-07-01 00:00:00             True   
3               145255     12827  2015-05-01 00:00:00             True   
4               145255     12827  2015-07-01 00:00:00             True   

  Citizenship          LegalType Title Language                 Bank  \
0              Close Corporation    Mr  English  First National Bank   
1              Close Corporation    Mr  English  First National Bank   
2              Close Corporation    Mr  English  First National Bank   
3              Close Corporation    Mr  English  First National Bank   
4              Close Corporation    Mr  English  First National Bank   

       AccountType  ...                    ExcessSelected CoverCategory  \
0  Current account  ...             Mobility - 

In [13]:
print(df['Province'].unique())


['Gauteng' 'KwaZulu-Natal' 'Mpumalanga' 'Eastern Cape' 'Western Cape'
 'Limpopo' 'North West' 'Free State' 'Northern Cape']


### Select Metrics (KPIs)

In [4]:
df['TotalClaims'] = df['TotalClaims']  # Target variable for claims
df['TotalPremium'] = df['TotalPremium']  # Target variable for premium

### Data Segmentation

In [25]:
group_a = df[df['Province'] == 'Mpumalanga']  # Control group
group_b = df[df['Province'] == 'KwaZulu-Natal']  # Test group

### Statistical Testing

#### For Risk Differences (Chi-Squared Test):

In [26]:
from scipy.stats import chi2_contingency

# Create a contingency table for the categorical feature (e.g., provinces)
contingency_table = pd.crosstab(df['Province'], df['TotalClaims'])

# Perform chi-squared test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Check if we reject the null hypothesis
if p_value < 0.05:
    print("Reject the null hypothesis: Significant risk difference across provinces")
else:
    print("Fail to reject the null hypothesis: No significant risk difference across provinces")

Fail to reject the null hypothesis: No significant risk difference across provinces


#### For Risk Differences Between Groups (T-Test):


In [27]:
print(f"Group A size: {len(group_a['TotalClaims'])}")
print(f"Group B size: {len(group_b['TotalClaims'])}")

Group A size: 52718
Group B size: 169781


In [28]:
from scipy.stats import ttest_ind

# T-test to compare means of TotalClaims between two groups
t_stat, p_value = ttest_ind(group_a['TotalClaims'], group_b['TotalClaims'])

# Check if we reject the null hypothesis
if p_value < 0.05:
    print("Reject the null hypothesis: Significant risk difference between groups")
else:
    print("Fail to reject the null hypothesis: No significant risk difference between groups")


Reject the null hypothesis: Significant risk difference between groups


###  Analyze and Report

In [30]:
# Example summary report
if p_value < 0.05:
    result = "Significant difference found"
else:
    result = "No significant difference found"

print(f"Analysis Result: {result}")
print(f"P-value: {p_value}")

Analysis Result: Significant difference found
P-value: 0.00030304013936754346
